In [1]:
# 使用gluon自定义计算层，使用ndarray实现，可以被重复调用

from mxnet import nd
from mxnet.gluon import nn


class CenteredLayer(nn.Block):

    def __init__(self, *args, **kwargs):
        super(CenteredLayer, self).__init__(*args, **kwargs)

    def forward(self, x):
        return x - x.mean()


layer = CenteredLayer()
layer.initialize()
layer(nd.array([1,2,3,4,5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [2]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128))
    net.add(nn.Dense(10))
    net.add(CenteredLayer())


net.initialize()
y = net(nd.random.uniform(shape=(4, 8)))

In [3]:
from mxnet import gluon

params = gluon.Parameter('exciting_parameter_yay', shape=(3,3))
params.initialize()

pd = gluon.ParameterDict(prefix='block1_')
pd.get('exciting_parameter_yay', shape=(3,3))

Parameter block1_exciting_parameter_yay (shape=(3, 3), dtype=<class 'numpy.float32'>)

In [4]:
class Dense(nn.Block):

    def __init__(self, units, in_units, *args, **kwargs):
        super(Dense, self).__init__(*args, **kwargs)
        with self.name_scope():
            self.weight = self.params.get('wegith', shape=(in_units, units))
            self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)


dense = Dense(5, in_units=10, prefix='o_my_dense')

dense.initialize()
dense(nd.random.uniform(shape=(2, 10)))


[[0.         0.         0.03019281 0.09594411 0.13613266]
 [0.         0.         0.00460232 0.10275272 0.15692511]]
<NDArray 2x5 @cpu(0)>

In [5]:
net = nn.Sequential()
with net.name_scope():
    net.add(Dense(32, in_units=64))
    net.add(Dense(2, in_units=32))

net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.         0.06250843]
 [0.00077505 0.08170695]]
<NDArray 2x2 @cpu(0)>